In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.sample import sample_gen

In [6]:
## Get Data 

df_ = pd.read_csv("/data/raw/Workshop_PUSO.csv", sep=';')
df_ = df_[['idsubsegmen_repair', 'latitude', 'longitude', 'obs', 'tahun', 'bulan',
       'kdprov', 'idsubfinal', 'idkab', 'idkec', 'idsegmen', 'is_puso']]
df_['geometry'] = df_.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf_ = gpd.GeoDataFrame(df_, geometry='geometry')
print(gdf_.shape)
gdf_.head(2)

(5399, 13)


,idsubsegmen_repair,latitude,longitude,obs,tahun,bulan,kdprov,idsubfinal,idkab,idkec,idsegmen,is_puso,geometry
0,320104008A1,-6.582085,106.646608,6.0,23,9,32,320104008A1239,3201,3201040,320104008,1,POINT (106.64661 -6.58209)
1,320104008A2,-6.582958,106.646563,6.0,23,9,32,320104008A2239,3201,3201040,320104008,1,POINT (106.64656 -6.58296)


### Get Elevation - Pixel Value

In [8]:
# Load Get Value From Raster

raster_path = "data geospasial lain/elevation_Jabar_100m.tif"  # Replace with your raster file
with rasterio.open(raster_path) as src:
    coordinates = [(geom.x, geom.y) for geom in gdf_.geometry]
    pixel_values = list(src.sample(coordinates))
band_names = ["elevation"] 
df_elevation = pd.DataFrame(pixel_values, columns=band_names)
df_elevation.head(2)

,elevation
0,237
1,240


In [10]:
# Load Get Value From Raster

raster_path = "data geospasial lain/landforms_Jabar_100m.tif"  # Replace with your raster file
with rasterio.open(raster_path) as src:
    coordinates = [(geom.x, geom.y) for geom in gdf_.geometry]
    pixel_values = list(src.sample(coordinates))
band_names = ["landform"] 
df_landform = pd.DataFrame(pixel_values, columns=band_names)
df_landform.head(2)

,landform
0,21
1,21


In [11]:
# Load Get Value From Raster

raster_path = "data geospasial lain/slope_Jabar_100m.tif"  # Replace with your raster file
with rasterio.open(raster_path) as src:
    coordinates = [(geom.x, geom.y) for geom in gdf_.geometry]
    pixel_values = list(src.sample(coordinates))
band_names = ["slope"] 
df_slope = pd.DataFrame(pixel_values, columns=band_names)
df_slope.head(2)

,slope
0,1.854334
1,1.315798


In [14]:
gdf_padi_ = pd.concat([gdf_, df_elevation, df_landform, df_slope], axis=1)
gdf_padi_.drop(['longitude','latitude','geometry'], axis='columns', inplace=True)
gdf_padi_.head(2)

,idsubsegmen_repair,obs,tahun,bulan,kdprov,idsubfinal,idkab,idkec,idsegmen,is_puso,elevation,landform,slope
0,320104008A1,6.0,23,9,32,320104008A1239,3201,3201040,320104008,1,237,21,1.854334
1,320104008A2,6.0,23,9,32,320104008A2239,3201,3201040,320104008,1,240,21,1.315798


In [15]:
gdf_padi_.to_csv("Data untuk modelling/data_compile_agro_geospasial_lain.csv")